In [20]:
from DbConnector import DbConnector
from decouple import config
from upload_data import get_all_user_data

In [8]:
connection = DbConnector(PASSWORD=config('USERPWD'))
client = connection.client
db = connection.db

You are connected to the database: test_db
-----------------------------------------------



In [21]:
users = get_all_user_data('dataset/dataset/Data')

NameError: name 'dataset_base_path' is not defined